In [1]:
import re
import os
import pandas as pd

def readfile(path):
    dirs = os.listdir(path)
    column_names = ['label', 'text']
    data_ = []
    data = []
    for d in dirs:
        if os.path.isdir(os.path.join(path, d)):
            path_d = path + d + '/'
            files = os.listdir(path_d)
            docs = ''
            for file in files:
                if os.path.isfile(os.path.join(path_d, file)):
                    doc = ''
                    f = open(os.path.join(path_d, file),'r')
                    for line in f:
                        if len(line)>0:
                            line = re.sub('[^a-zA-Z]', ' ', line)
                            words = []
                            for word in line.split():
                                if len(word)>1:
                                    word = word.strip().lower()
                                    words.append(word)
                            line = ' '.join(words)
                            doc = doc + " " + line
                row = [d, doc.strip()]
                docs = docs + " " + doc
                data.append(row)
            row_ = [d, docs.strip()]
            data_.append(row_)
    df_ = pd.DataFrame(data_, columns=column_names)
    df = pd.DataFrame(data, columns=column_names)
    return df_, df
                            
path_train = 'C50/C50train/'
path_test = 'C50/C50test/'   
df_train_, df_train = readfile(path_train)
df_test_, df_test = readfile(path_test)
         

In [2]:
df_train_.shape, df_train.shape, df_test_.shape, df_test.shape

((50, 2), (2500, 2), (50, 2), (2500, 2))

In [3]:
len(df_train_.iloc[0, 1]), len(df_train.iloc[0, 1]), len(df_test_.iloc[0, 1]), len(df_test.iloc[0,1])

(141649, 3948, 144347, 2399)

In [4]:
df_train_[:3]

,label,text
0,RobinSidel,drugstore giant revco inc said monday it agree...
1,LynnleyBrowning,big diamond deal between russia and de beers h...
2,KouroshKarimkhany,apple computer inc friday agreed to acquire ne...


In [5]:
df_train[:3]

,label,text
0,RobinSidel,drugstore giant revco inc said monday it agree...
1,RobinSidel,mattel inc seeking to expand in the market for...
2,RobinSidel,financial agreement between barney inc and hon...


In [6]:
df_test_[:3]

,label,text
0,RobinSidel,itt corp met with financial advisers on thursd...
1,LynnleyBrowning,russia norilsk nickel an arctic industrial met...
2,KouroshKarimkhany,apple computer inc beleaguered by rivals offer...


In [7]:
df_test[:3]

,label,text
0,RobinSidel,itt corp met with financial advisers on thursd...
1,RobinSidel,norfolk southern corp on monday launched new e...
2,RobinSidel,long troubled solid waste firm mid american wa...


In [8]:
labels = list(df_train_['label'])
print(labels)

['RobinSidel', 'LynnleyBrowning', 'KouroshKarimkhany', 'MichaelConnor', 'JoeOrtiz', 'EricAuchard', 'AaronPressman', 'SimonCowell', "LynneO'Donnell", 'EdnaFernandes', 'KevinMorrison', 'SamuelPerry', 'PatriciaCommins', 'JohnMastrini', 'JanLopatka', 'KevinDrawbaugh', 'KarlPenhaul', 'MartinWolk', 'ScottHillis', 'DavidLawder', 'FumikoFujisaki', 'MarcelMichelson', 'NickLouth', 'DarrenSchuettler', 'WilliamKazer', 'TanEeLyn', 'PierreTran', 'HeatherScoffield', 'MureDickie', 'RogerFillion', 'JimGilchrist', 'BradDorfman', 'AlanCrosby', 'JonathanBirt', 'BenjaminKangLim', 'TheresePoletti', 'KeithWeir', 'JoWinterbottom', 'MarkBendeich', 'JaneMacartney', 'MatthewBunce', 'ToddNissen', 'PeterHumphrey', 'TimFarrand', 'SarahDavison', 'GrahamEarnshaw', 'BernardHickey', 'KirstinRidley', 'AlexanderSmith', 'LydiaZajc']


In [9]:
X_train = df_train['text']
y_train = df_train['label']
X_test = df_test['text']
y_test = df_test['label']


In [10]:
# way 1, simple SVM

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import confusion_matrix

vectorizer = CountVectorizer(binary=True)
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)


svm = svm.SVC(kernel = 'linear', probability=True)
prob = svm.fit(X_train_vec, y_train).predict_proba(X_test_vec)
y_pred_svm = svm.predict(X_test_vec)


print("Accuracy:", accuracy_score(y_test, y_pred_svm))

Accuracy: 0.7156


In [12]:
# Accuracy: 0.7156

In [13]:
print(pd.DataFrame(confusion_matrix(y_test, y_pred_svm)).loc[10:25, 10:25])

    10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25
10  49   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11   0  45   0   0   0   0   0   0   0   0   0   0   0   0   0   0
12   0   1  21   4   0   0   0   0   3   0   3   0   0   0   0   0
13   0   0   0  30   0   0   0   0  13   0   0   0   0   0   0   0
14   0   2   0   0  17   0   0   0   0   0   0   0   0   0   0   0
15   0   0   0   0   0  50   0   0   0   0   0   0   0   0   0   0
16   0   0   0   0   0   0  39   0   0   2   0   1   0   0   5   0
17   0   0   0   0   0   0   0  32   0   2   0   0   0   0   0   0
18   0   0   0  14   0   0   0   0  35   0   0   0   0   0   0   0
19   0   0   0   0   0   0   1   0   0  40   0   1   0   0   0   0
20   0   0   0   0   0   0   0   0   0   0  50   0   0   0   0   0
21   0   0   0   0   0   0   0   0   0   0   0  44   0   0   0   0
22   0   0   0   0   0   0   0   0   0   1   0   0  32   0   0   0
23   0   0   0   0   2   0   0   0   0   0   0   0   0  39   0

In [14]:
print("normalized_mutual_info_score between predict and test:", normalized_mutual_info_score(y_test, y_pred_svm))


normalized_mutual_info_score between predict and test: 0.7693455752245238
